In [6]:
# importing the libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import copy
#from tqdm import tqdm, tqdm_notebook, trange
from scipy.optimize import fmin
from IPython.display import clear_output

#import lightgbm as lgbm

from sklearn.metrics import f1_score, roc_auc_score, log_loss, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier

#from eli5.permutation_importance import get_score_importances

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [8]:
class TrainTestHelper(object):
    def __init__(self):
        self.ntrain = None

    def combine(self, train, test):
        self.ntrain = train.shape[0]
        if isinstance(train, np.ndarray):
            return np.row_stack((train, test))
        else:
            return train.append(test).reset_index(drop=True)

    def split(self, train_test):
        if self.ntrain is None:
            return None
        if isinstance(train_test, np.ndarray):
            train = train_test[:self.ntrain, :]
            test = train_test[self.ntrain:, :]
        else:
            train = train_test.iloc[:self.ntrain, :].copy().reset_index(drop=True)
            test = train_test.iloc[self.ntrain:, :].copy().reset_index(drop=True)
        return train, test

In [ ]:
class DataDivision(object):
    """
    this is a helper class that divides the data into trainset and validset and vice versa
    """
    def __init__(self):
        self.xshape = None
        
    def concat(self, train, valid):
        """
        this module combines the already divided trainset and validset into one whole dataset 
        """
        self.xshape = train.shape[0]  # len of the trainset
        
        # below logic checks whether train is numpy instance or pandas instance and deals with it accordingly
        if isinstance(train, np.ndarray):
            return np.row_stack((train, valid))
        else:
            return train.append(valid).reset_index(drop = True)
        
    def split(self, train_test):
        if self.xshape is None:
            return None
        